In [2]:
!pip install -q "transformers>=4.40.0" "datasets==3.6.0" "accelerate>=0.30.0"
# !pip install --upgrade datasets
# !pip -q install -U \
#   "numpy==2.0.2" \
#   "pandas==2.2.2" \
#   "datasets==3.6.0" \
#   "transformers>=4.40.0" \
#   accelerate evaluate scikit-learn

0: Politics   (정치 혐오)
1: Origin     (출신/지역/학벌 등)
2: Physical   (외모/신체, 장애 등)
3: Age        (연령/세대)
4: Gender     (성별)
5: Religion   (종교)
6: Race       (인종)
7: Profanity  (욕설/비속어)
8: Not Hate Speech (혐오 아님)

In [3]:
from transformers import AutoTokenizer, AutoConfig, AutoModelForSequenceClassification
import torch

base_model_name = "beomi/KcELECTRA-base-v2022"

# KMHaS에서 사용할 혐오 유형 8개 (순서 고정)
label_list = [
    "Politics",   # 0
    "Origin",     # 1
    "Physical",   # 2
    "Age",        # 3
    "Gender",     # 4
    "Religion",   # 5
    "Race",       # 6
    "Profanity",  # 7
]
num_labels = len(label_list)

id2label = {i: name for i, name in enumerate(label_list)}
label2id = {name: i for i, name in enumerate(label_list)}

config = AutoConfig.from_pretrained(
    base_model_name,
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id,
    problem_type="multi_label_classification",  # 핵심: 멀티라벨로 설정
)

tokenizer = AutoTokenizer.from_pretrained(base_model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    base_model_name,
    config=config,
)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

print("Device:", device)
print("Labels:", id2label)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/504 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/288 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/511M [00:00<?, ?B/s]

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base-v2022 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model.safetensors:   0%|          | 0.00/511M [00:00<?, ?B/s]

Device: cuda
Labels: {0: 'Politics', 1: 'Origin', 2: 'Physical', 3: 'Age', 4: 'Gender', 5: 'Religion', 6: 'Race', 7: 'Profanity'}


In [4]:
import torch
import numpy as np

def predict_multilabel(texts, threshold=0.5):
    """
    texts: 문자열 리스트
    threshold: 이 값 이상이면 '1(해당 혐오 있음)'으로 판단
    """
    model.eval()
    results = []

    # 배치 처리 (여기서는 간단히 한 번에 처리)
    enc = tokenizer(
        texts,
        padding=True,
        truncation=True,
        max_length=128,
        return_tensors="pt",
    ).to(device)

    with torch.no_grad():
        outputs = model(**enc)
        logits = outputs.logits  # shape: (batch, num_labels)
        probs = torch.sigmoid(logits).cpu().numpy()  # 0~1 확률

    for i, text in enumerate(texts):
        p = probs[i]  # 길이 8짜리 확률 벡터
        # threshold 기준으로 0/1 예측
        pred_binary = (p >= threshold).astype(int)

        # 1로 예측된 라벨 이름들 모으기
        active_labels = [label_list[j] for j, v in enumerate(pred_binary) if v == 1]

        print("문장:", text)
        print("확률 벡터:", np.round(p, 4))  # 보기 좋게 반올림
        print("1로 예측된 라벨:", active_labels if active_labels else "None (Non-hate로 볼 수 있음)")
        print("-" * 60)

        results.append({
            "text": text,
            "probs": p,
            "pred_binary": pred_binary,
            "active_labels": active_labels,
        })

    return results


In [5]:
texts = [
    "오늘 날씨가 정말 좋다.",                         # Non-hate 예상
    "이민자들은 다 쫓아내야 한다.",                   # Race/Politics 계열 혐오
    "그 종교 믿는 사람들은 전부 멍청이야.",           # Religion + Profanity 가능
    "여자들은 운전 못해서 사고만 낸다.",              # Gender 혐오
    "야 너 진짜 개같이 생겼다.",                       # Physical + Profanity
]

_ = predict_multilabel(texts, threshold=0.5)


문장: 오늘 날씨가 정말 좋다.
확률 벡터: [0.4876 0.4837 0.507  0.5101 0.4797 0.4867 0.5169 0.4738]
1로 예측된 라벨: ['Physical', 'Age', 'Race']
------------------------------------------------------------
문장: 이민자들은 다 쫓아내야 한다.
확률 벡터: [0.4751 0.4879 0.5007 0.5007 0.4734 0.5019 0.524  0.4715]
1로 예측된 라벨: ['Physical', 'Age', 'Religion', 'Race']
------------------------------------------------------------
문장: 그 종교 믿는 사람들은 전부 멍청이야.
확률 벡터: [0.4931 0.474  0.5025 0.4835 0.5054 0.4837 0.4985 0.469 ]
1로 예측된 라벨: ['Physical', 'Gender']
------------------------------------------------------------
문장: 여자들은 운전 못해서 사고만 낸다.
확률 벡터: [0.4888 0.4786 0.4693 0.5301 0.47   0.4936 0.5401 0.4802]
1로 예측된 라벨: ['Age', 'Race']
------------------------------------------------------------
문장: 야 너 진짜 개같이 생겼다.
확률 벡터: [0.4903 0.4839 0.4925 0.5222 0.4728 0.5005 0.5072 0.4739]
1로 예측된 라벨: ['Age', 'Religion', 'Race']
------------------------------------------------------------


# 4. Fine-tuning 환경 세팅 (KMHaS + Multi-label)
4-1. 이 단계의 목표

이 단계에서 끝내고 싶은 것:

Colab에 필요한 라이브러리 설치

KMHaS 데이터셋 로드

KMHaS의 label(클래스 인덱스 리스트)을
→ 길이 8짜리 “멀티라벨 벡터”로 변환 (Origin~Religion, Not Hate는 제외)

텍스트 토크나이징

Trainer가 바로 학습을 시작할 수 있을 정도까지
TrainingArguments, Trainer 골격 세팅

In [6]:
!pip install -q "evaluate>=0.4.1"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.1 MB/s eta 0:00:00


In [7]:
import numpy as np
import torch

from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer,
)
import evaluate

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

hate_labels = [
    "origin",    # 0
    "physical",  # 1
    "politics",  # 2
    "profanity", # 3
    "age",       # 4
    "gender",    # 5
    "race",      # 6
    "religion",  # 7
]
num_labels = 8



In [8]:
dataset = load_dataset("jeanlee/kmhas_korean_hate_speech")
print(dataset)

print("Train example 0:")
print(dataset["train"][0])

print("Features:")
print(dataset["train"].features)


README.md: 0.00B [00:00, ?B/s]

kmhas_korean_hate_speech.py: 0.00B [00:00, ?B/s]

default/train/0000.parquet:   0%|          | 0.00/5.24M [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/579k [00:00<?, ?B/s]

default/test/0000.parquet:   0%|          | 0.00/1.46M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/78977 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/8776 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/21939 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 78977
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 8776
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 21939
    })
})
Train example 0:
{'text': '"자한당틀딱들.. 악플질 고만해라."', 'label': [2, 4]}
Features:
{'text': Value(dtype='string', id=None), 'label': Sequence(feature=ClassLabel(names=['origin', 'physical', 'politics', 'profanity', 'age', 'gender', 'race', 'religion', 'not_hate_speech'], id=None), length=-1, id=None)}


In [9]:
# KMHaS 원본 순서 기반 혐오 유형 8개
hate_labels = [
    "origin",    # 0
    "physical",  # 1
    "politics",  # 2
    "profanity", # 3
    "age",       # 4
    "gender",    # 5
    "race",      # 6
    "religion",  # 7
]
num_labels = len(hate_labels)

id2label = {i: name for i, name in enumerate(hate_labels)}
label2id = {name: i for i, name in enumerate(hate_labels)}

def kmhas_to_multilabel(example):
    """
    KMHaS의 label(list[int])을 길이 8짜리 0.0/1.0 float 벡터로 변환.
    8(not hate)는 벡터에 반영하지 않음 → 전부 0.0이면 non-hate로 해석.
    """
    vec = [0.0] * num_labels  # <== float로 초기화

    for idx in example["label"]:
        if idx == 8:
            # not hate speech → 아무 것도 켜지지 않은 상태
            continue
        if 0 <= idx < 8:
            vec[idx] = 1.0      # <== float로 저장

    example["labels"] = vec
    return example

dataset_ml = dataset.map(kmhas_to_multilabel)
print("After kmhas_to_multilabel:")
print(dataset_ml["train"][0])
print("labels:", dataset_ml["train"][0]["labels"], type(dataset_ml["train"][0]["labels"][0]))

Map:   0%|          | 0/78977 [00:00<?, ? examples/s]

Map:   0%|          | 0/8776 [00:00<?, ? examples/s]

Map:   0%|          | 0/21939 [00:00<?, ? examples/s]

After kmhas_to_multilabel:
{'text': '"자한당틀딱들.. 악플질 고만해라."', 'label': [2, 4], 'labels': [0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0]}
labels: [0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0] <class 'float'>


In [10]:
base_model_name = "beomi/KcELECTRA-base-v2022"

config = AutoConfig.from_pretrained(
    base_model_name,
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id,
    problem_type="multi_label_classification",  # 멀티라벨 설정 핵심
)

tokenizer = AutoTokenizer.from_pretrained(base_model_name)

model = AutoModelForSequenceClassification.from_pretrained(
    base_model_name,
    config=config,
)

model.to(device)

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base-v2022 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(54343, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): L

In [11]:
import torch

def tokenize_and_align(batch):
    enc = tokenizer(
        batch["text"],
        truncation=True,
        max_length=128,
    )
    # labels를 float32로 캐스팅해서 넘겨줌
    enc["labels"] = [np.array(l, dtype="float32") for l in batch["labels"]]
    return enc

tokenized = {}

for split in ["train", "validation", "test"]:
    tokenized[split] = dataset_ml[split].map(
        tokenize_and_align,
        batched=True,
    )
    # Remove set_format calls here; DataCollator will handle tensor conversion.
    # For this approach, we do NOT want to set the format to 'torch' here,
    # as it might interfere with the custom data collator's type handling.
    pass # No set_format call for now

# Trainer가 필요 없는 컬럼 제거
keep_cols = ["input_ids", "attention_mask", "labels"]

for split in ["train", "validation", "test"]:
    cols_to_remove = [c for c in tokenized[split].column_names if c not in keep_cols]
    tokenized[split] = tokenized[split].remove_columns(cols_to_remove)

print("tokenized[\"train\"] example:")
print(tokenized["train"][0])
print(type(tokenized["train"][0]["labels"]), tokenized["train"][0]["labels"])

Map:   0%|          | 0/78977 [00:00<?, ? examples/s]

Map:   0%|          | 0/8776 [00:00<?, ? examples/s]

Map:   0%|          | 0/21939 [00:00<?, ? examples/s]

tokenized["train"] example:
{'labels': [0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0], 'input_ids': [2, 6, 8535, 15271, 4079, 18, 18, 10377, 4077, 27100, 18, 6, 3], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
<class 'list'> [0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0]


In [12]:
import torch
from transformers import DataCollatorWithPadding

# Custom Data Collator to ensure labels are torch.float32
class CustomDataCollator(DataCollatorWithPadding):
    def __call__(self, features):
        # Pad input_ids and attention_mask
        # This will convert them to torch.LongTensor by default
        batch = self.tokenizer.pad(features, return_tensors="pt")

        # Extract labels and convert them to torch.float32
        # 'features' is a list of dictionaries, where each dict is an example.
        # Each example's 'labels' is a Python list of floats from tokenize_and_align.
        labels = [feature["labels"] for feature in features]
        batch["labels"] = torch.tensor(labels, dtype=torch.float32)

        return batch

data_collator = CustomDataCollator(tokenizer=tokenizer)

metric_f1 = evaluate.load("f1")
metric_acc = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred  # logits: (batch, 8), labels: (batch, 8)
    probs = 1 / (1 + np.exp(-logits))       # sigmoid
    preds = (probs >= 0.5).astype(int)      # threshold=0.5

    # 멀티라벨이므로 2차원 → 1차원으로 펼쳐서 계산
    preds_flat = preds.reshape(-1)
    labels_flat = labels.reshape(-1)

    f1_micro = metric_f1.compute(
        predictions=preds_flat,
        references=labels_flat,
        average="micro",
    )["f1"]

    f1_macro = metric_f1.compute(
        predictions=preds_flat,
        references=labels_flat,
        average="macro",
    )["f1"]

    acc = metric_acc.compute(
        predictions=preds_flat,
        references=labels_flat,
    )["accuracy"]

    return {
        "f1_micro": f1_micro,
        "f1_macro": f1_macro,
        "accuracy": acc,
    }

In [13]:
training_args = TrainingArguments(
    output_dir="./kmhas_multilabel_debug",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=1,         # 5단계에서 바꿀 예정
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="no",
    logging_steps=50,
    load_best_model_at_end=False,
    report_to="none",           # wandb 등 안 씀
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer


/tmp/ipython-input-1360801119.py:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


# 5. 소량 데이터로 Fine-tuning 시도 (오버피팅 테스트)

5단계 완료 체크리스트

1. train_small, valid_small를 만들어서 길이를 확인했다.

2. small_model, small_trainer를 만들어 train()을 돌려봤다.

3. epoch이 진행될수록 train loss가 줄고, f1_micro/f1_macro가 눈에 띄게 올라가는 걸 확인했다.

4. “이제 full data로 학습해도 되겠다”는 감각이 들었다.

In [14]:
# 소량 데이터 subset (숫자는 필요에 따라 조정 가능)
train_small = tokenized["train"].select(range(1000))
valid_small = tokenized["validation"].select(range(200))

len(train_small), len(valid_small)


(1000, 200)

In [15]:
from transformers import TrainingArguments, Trainer

# 멀티라벨 설정은 4단계와 동일
small_model = AutoModelForSequenceClassification.from_pretrained(
    base_model_name,
    config=config,
).to(device)

small_training_args = TrainingArguments(
    output_dir="./kmhas_multilabel_small_overfit",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=5,        # 오버피팅 확인을 위해 5 epoch 정도
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="no",
    logging_steps=20,
    load_best_model_at_end=False,
    report_to="none",          # wandb 등 사용 안 함
)

small_trainer = Trainer(
    model=small_model,
    args=small_training_args,
    train_dataset=train_small,
    eval_dataset=valid_small,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

small_trainer


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base-v2022 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-2439570069.py:23: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  small_trainer = Trainer(


In [16]:
small_train_result = small_trainer.train()
small_train_result

small_eval = small_trainer.evaluate(valid_small)
small_eval


You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,F1 Micro,F1 Macro,Accuracy
1,0.601500,0.396825,0.935625,0.483371,0.935625
2,0.346600,0.294634,0.935625,0.483371,0.935625
3,0.307700,0.263599,0.935625,0.483371,0.935625
4,0.265200,0.251921,0.935625,0.483371,0.935625
5,0.266300,0.249103,0.935625,0.483371,0.935625


{'eval_loss': 0.24910281598567963,
 'eval_f1_micro': 0.935625,
 'eval_f1_macro': 0.4833710041976106,
 'eval_accuracy': 0.935625,
 'eval_runtime': 0.8149,
 'eval_samples_per_second': 245.442,
 'eval_steps_per_second': 8.59,
 'epoch': 5.0}

# 6. Full data로 본격 학습 (KMHaS + Multi-label)

In [17]:
train_full = tokenized["train"]
valid_full = tokenized["validation"]

len(train_full), len(valid_full)

(78977, 8776)

In [18]:
from transformers import TrainingArguments, Trainer

# full data 학습용 모델 (매번 새로 초기화)
full_model = AutoModelForSequenceClassification.from_pretrained(
    base_model_name,
    config=config,
).to(device)

full_training_args = TrainingArguments(
    output_dir="./kmhas_multilabel_full_v1",  # 버전명 v1
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=3,              # 2~3 정도가 현실적
    weight_decay=0.01,
    eval_strategy="epoch",     # epoch 끝날 때마다 평가
    save_strategy="epoch",           # epoch마다 체크포인트 저장
    logging_steps=200,               # 로그 간격 (데이터 크기에 따라 조정 가능)
    load_best_model_at_end=True,     # 가장 좋은 eval 성능의 모델 로드
    metric_for_best_model="f1_macro",# 어떤 지표 기준으로 "best"를 볼지
    greater_is_better=True,
    report_to="none",                # wandb 등 사용 안 함
)

full_trainer = Trainer(
    model=full_model,
    args=full_training_args,
    train_dataset=train_full,
    eval_dataset=valid_full,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

full_trainer


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base-v2022 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-3756053209.py:25: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  full_trainer = Trainer(


In [21]:
full_train_result = full_trainer.train()
full_train_result

Epoch,Training Loss,Validation Loss,F1 Micro,F1 Macro,Accuracy
1,0.064600,0.064430,0.975786,0.914273,0.975786
2,0.050200,0.059270,0.977951,0.920564,0.977951
3,0.038900,0.058336,0.979561,0.925379,0.979561


TrainOutput(global_step=7407, training_loss=0.06341298457838125, metrics={'train_runtime': 2661.1702, 'train_samples_per_second': 89.033, 'train_steps_per_second': 2.783, 'total_flos': 7756664017186896.0, 'train_loss': 0.06341298457838125, 'epoch': 3.0})

In [22]:
full_eval = full_trainer.evaluate(valid_full)
full_eval


{'eval_loss': 0.05833601951599121,
 'eval_f1_micro': 0.9795607338195077,
 'eval_f1_macro': 0.9253788279093667,
 'eval_accuracy': 0.9795607338195077,
 'eval_runtime': 31.83,
 'eval_samples_per_second': 275.715,
 'eval_steps_per_second': 8.64,
 'epoch': 3.0}

# 1차 baseline 모델 저장

In [23]:
save_dir = "./kmhas_multilabel_full_v1/best_model"

full_trainer.model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)

save_dir

'./kmhas_multilabel_full_v1/best_model'

# 7. Hyper-parameter Tuning (KMHaS Multi-label)

공통 Trainer 생성 함수 만들기

In [19]:
def make_trainer(run_name, learning_rate, num_epochs, batch_size):
    model = AutoModelForSequenceClassification.from_pretrained(
        base_model_name,
        config=config,
    ).to(device)

    args = TrainingArguments(
        output_dir=f"./kmhas_multilabel_{run_name}",
        learning_rate=learning_rate,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=num_epochs,
        weight_decay=0.01,
        eval_strategy="epoch",
        save_strategy="epoch",
        logging_steps=200,
        load_best_model_at_end=True,
        metric_for_best_model="f1_macro",
        greater_is_better=True,
        report_to="none",
    )

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=train_full,
        eval_dataset=valid_full,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )

    return trainer

# v2 / v3 실험 돌리기

# v2 | 학습률만 살짝 키워보기

목표: “조금 더 공격적으로 학습하면 좋아질까?”

설정: lr=3e-5, epoch=3, batch=32

기대: 학습 속도는 빨라질 수 있지만, 너무 크면 오히려 성능이 떨어질 수도 있음

v1보다 eval_f1_macro가 좋아지는지/나빠지는지 비교

In [24]:
# v2: lr만 키운 버전
trainer_v2 = make_trainer(
    run_name="full_v2_lr3e-5_ep3_bs32",
    learning_rate=3e-5,
    num_epochs=3,
    batch_size=32,
)
train_result_v2 = trainer_v2.train()
eval_result_v2 = trainer_v2.evaluate(valid_full)
print("v2 eval:", eval_result_v2)

# v2 모델과 토크나이저 저장
save_dir_v2 = "./kmhas_multilabel_full_v2_lr3e-5_ep3_bs32/best_model"
trainer_v2.model.save_pretrained(save_dir_v2)
tokenizer.save_pretrained(save_dir_v2)

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base-v2022 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-3929312374.py:23: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,F1 Micro,F1 Macro,Accuracy
1,0.064100,0.064121,0.975829,0.914387,0.975829
2,0.049100,0.059700,0.978421,0.922170,0.978421
3,0.034900,0.060148,0.979518,0.924830,0.979518


v2 eval: {'eval_loss': 0.06014769896864891, 'eval_f1_micro': 0.9795180036463081, 'eval_f1_macro': 0.9248295930720527, 'eval_accuracy': 0.9795180036463081, 'eval_runtime': 31.1467, 'eval_samples_per_second': 281.764, 'eval_steps_per_second': 8.829, 'epoch': 3.0}


('./kmhas_multilabel_full_v2_lr3e-5_ep3_bs32/best_model/tokenizer_config.json',
 './kmhas_multilabel_full_v2_lr3e-5_ep3_bs32/best_model/special_tokens_map.json',
 './kmhas_multilabel_full_v2_lr3e-5_ep3_bs32/best_model/vocab.txt',
 './kmhas_multilabel_full_v2_lr3e-5_ep3_bs32/best_model/added_tokens.json',
 './kmhas_multilabel_full_v2_lr3e-5_ep3_bs32/best_model/tokenizer.json')

# v3 | epoch만 늘려보기

목표: “조금 더 오래 학습하면 더 잘 배울까?”

설정: lr=2e-5, epoch=4, batch=32

기대:

train_loss는 더 내려갈 것

valid 성능은 올라가다가, 너무 오래 학습하면 과적합으로 떨어질 수도 있음

In [ ]:
# v3: epoch만 늘린 버전
trainer_v3 = make_trainer(
    run_name="full_v3_lr2e-5_ep4_bs32",
    learning_rate=2e-5,
    num_epochs=4,
    batch_size=32,
)
train_result_v3 = trainer_v3.train()
eval_result_v3 = trainer_v3.evaluate(valid_full)
print("v3 eval:", eval_result_v3)

# v3 모델과 토크나이저 저장 (필요시)
save_dir_v3 = "./kmhas_multilabel_full_v3_lr2e-5_ep4_bs32/best_model"
trainer_v3.model.save_pretrained(save_dir_v3)
tokenizer.save_pretrained(save_dir_v3)


# 멀티라벨 혼동 행렬(Confusion Matrix)
v2 모델을 기준

우리가 혼동행렬로 확인하고 싶은 것:

“각 혐오 유형별로
이 모델이 얼마나 잘 맞추고,
어디서 많이 헷갈리는지(놓치거나, 과하게 찍는지)”

FN: “혐오를 놓치는 것” (안전·윤리 관점에서 민감)
FP: “혐오 아닌데 혐오로 보는 것” (사용자 경험 관점에서 민감)

| 실제/예측 | 0 (혐오 아님)           | 1 (혐오 있음)           |
| ----- | ------------------- | ------------------- |
| 실제 0  | TN (True Negative)  | FP (False Positive) |
| 실제 1  | FN (False Negative) | TP (True Positive)  |

*특히 소수자 혐오(gender, race, religion 등)에서 FN/FP가 어떻게 나오는지 확인 필요


In [ ]:
import numpy as np

THRESHOLD = 0.5  # 일단 0.5 기준으로 0/1 자르기

pred_output = trainer_v2.predict(valid_full)

logits = pred_output.predictions   # (N, 8) 각 혐오 유형에 대한 점수
labels = pred_output.label_ids     # (N, 8) 정답 0/1 벡터

# sigmoid로 0~1 확률로 바꾸기
probs = 1 / (1 + np.exp(-logits))

# threshold 기준으로 0/1 예측 만들기
preds = (probs >= THRESHOLD).astype(int)  # (N, 8)


In [ ]:
from sklearn.metrics import multilabel_confusion_matrix

# labels, preds: shape (N, 8)
cm_per_label = multilabel_confusion_matrix(labels, preds)
cm_per_label.shape  # (8, 2, 2)


(8, 2, 2)

In [ ]:
hate_labels = [
    "origin",    # 0
    "physical",  # 1
    "politics",  # 2
    "profanity", # 3
    "age",       # 4
    "gender",    # 5
    "race",      # 6
    "religion",  # 7
]

for i, label_name in enumerate(hate_labels):
    tn, fp, fn, tp = cm_per_label[i].ravel()

    print(f"===== Label: {label_name} =====")
    print(f"TP (실제 1, 예측 1: 혐오를 제대로 잡음)       : {tp}")
    print(f"FN (실제 1, 예측 0: 혐오를 놓침)             : {fn}")
    print(f"FP (실제 0, 예측 1: 아닌데 혐오라고 봄)      : {fp}")
    print(f"TN (실제 0, 예측 0: 혐오 아님을 잘 맞춤)     : {tn}")
    print()


===== Label: origin =====
TP (실제 1, 예측 1: 혐오를 제대로 잡음)       : 638
FN (실제 1, 예측 0: 혐오를 놓침)             : 121
FP (실제 0, 예측 1: 아닌데 혐오라고 봄)      : 130
TN (실제 0, 예측 0: 혐오 아님을 잘 맞춤)     : 7887

===== Label: physical =====
TP (실제 1, 예측 1: 혐오를 제대로 잡음)       : 632
FN (실제 1, 예측 0: 혐오를 놓침)             : 85
FP (실제 0, 예측 1: 아닌데 혐오라고 봄)      : 136
TN (실제 0, 예측 0: 혐오 아님을 잘 맞춤)     : 7923

===== Label: politics =====
TP (실제 1, 예측 1: 혐오를 제대로 잡음)       : 800
FN (실제 1, 예측 0: 혐오를 놓침)             : 96
FP (실제 0, 예측 1: 아닌데 혐오라고 봄)      : 163
TN (실제 0, 예측 0: 혐오 아님을 잘 맞춤)     : 7717

===== Label: profanity =====
TP (실제 1, 예측 1: 혐오를 제대로 잡음)       : 1225
FN (실제 1, 예측 0: 혐오를 놓침)             : 66
FP (실제 0, 예측 1: 아닌데 혐오라고 봄)      : 91
TN (실제 0, 예측 0: 혐오 아님을 잘 맞춤)     : 7394

===== Label: age =====
TP (실제 1, 예측 1: 혐오를 제대로 잡음)       : 497
FN (실제 1, 예측 0: 혐오를 놓침)             : 55
FP (실제 0, 예측 1: 아닌데 혐오라고 봄)      : 116
TN (실제 0, 예측 0: 혐오 아님을 잘 맞춤)     : 8108

===== Label: gender =====
TP (실제 1, 예측 1: 혐오를 제대로 잡음)       

TP (True Positive)
실제로 그 유형의 혐오가 “있는데”, 모델도 1(있음)이라고 맞춘 개수

FN (False Negative)
실제로 혐오가 “있는데”, 모델이 0(없음)이라고 해서 놓친 개수

FP (False Positive)
실제로 혐오가 “없는데”, 모델이 1(있음)이라고 과하게 본 개수

TN (True Negative)
실제로 혐오가 “없고”, 모델도 0(없음)이라고 맞춘 개수

# 결과 분석

| 라벨        | 실제 혐오(1)| Precision | Recall | F1   |
| ---------   | ----------- | --------- | ------ | ---- |
| origin      | 759         | 0.83      | 0.84   | 0.84 |
| physical    | 717         | 0.82      | 0.88   | 0.85 |
| politics    | 896         | 0.83      | 0.89   | 0.86 |
| profanity   | 1291        | 0.93      | 0.95   | 0.94 |
| age         | 552         | 0.81      | 0.90   | 0.85 |
| gender      | 659         | 0.78      | 0.80   | 0.79 |
| race        | 24          | 0.53      | 0.42   | 0.47 |
| religion    | 140         | 0.79      | 0.91   | 0.85 |

# 특이사항

gender: FN도 좀 있고 FP도 조금 많은 편이라,“놓치는 것도 있고, 아닌데 혐오라고 보는 경우도 있다.”

사회적으로 민감한 라벨이기 때문에,이 부분은 향후 데이터 보강이나 threshold 조정 등으로 좀 더 신중하게 다듬어야 할 후보

race: 인종/이주민 혐오를 탐지하는 능력은 다른 라벨에 비해 가장 약하고, 데이터 자체도 너무 적어서 추가 보강이 필요(데이터 한계)

# 최종 정리
멀티라벨 혼동행렬 분석 결과,욕설·정치·외모·나이·종교 혐오에 대해서는
F1 0.84~0.94 수준으로 비교적 안정적인 탐지 성능을 보였습니다.
반면, 성별 혐오(gender)는 F1이 약 0.79로 다소 낮고,
인종/이주민 혐오(race)는 실제 라벨 수가 매우 적어(Few-shot 수준)
F1이 약 0.47에 머무르는 등, 소수자 관련 혐오(특히 인종)에 대해서는
데이터 보강과 추가적인 모델 개선이 필요하다는 한계를 확인했습니다.”

# 오분류 사례 & 소수자 슬라이스 시연

In [ ]:
# valid_full에서 원문 텍스트를 가져온다고 가정
# (dataset을 tokenize하기 전에 text를 따로 저장했거나,
#  valid_full에 text 컬럼이 남아있으면 사용 가능)
texts = dataset_ml["validation"]["text"]  # 필요에 따라 수정

idx_gender = hate_labels.index("gender")
y_true_gender = labels[:, idx_gender]
y_pred_gender = preds[:, idx_gender]

# FN: 실제 1, 예측 0
fn_indices = np.where((y_true_gender == 1) & (y_pred_gender == 0))[0]
# FP: 실제 0, 예측 1
fp_indices = np.where((y_true_gender == 0) & (y_pred_gender == 1))[0]

print("=== Gender FN 예시 ===")
for i in fn_indices[:3]:
    print(f"[문장] {texts[i]}")
    print(f" - 실제 gender 라벨: 1 (혐오 있음)")
    print(f" - 모델 예측 gender 라벨: 0 (혐오 없음)")
    print()

print("=== Gender FP 예시 ===")
for i in fp_indices[:3]:
    print(f"[문장] {texts[i]}")
    print(f" - 실제 gender 라벨: 0 (혐오 없음)")
    print(f" - 모델 예측 gender 라벨: 1 (혐오 있음)")
    print()


=== Gender FN 예시 ===
[문장] 판사= 남자최종범=남자남자는 남자편인가.그래서 기각됫나.
 - 실제 gender 라벨: 1 (혐오 있음)
 - 모델 예측 gender 라벨: 0 (혐오 없음)

[문장] 뚜뚱하뇬들 극혐!!오로지 쳐먹는것!
 - 실제 gender 라벨: 1 (혐오 있음)
 - 모델 예측 gender 라벨: 0 (혐오 없음)

[문장] "여자들끼리 안질려고 더 이쁘게 해놓고 이제와선 남자들때문에 화장성형했다 ㅇㅈㄹ..."
 - 실제 gender 라벨: 1 (혐오 있음)
 - 모델 예측 gender 라벨: 0 (혐오 없음)

=== Gender FP 예시 ===
[문장] 접촉사고 문제가 아니라 신호위반이 문제야 빠순이들 정신 차려라
 - 실제 gender 라벨: 0 (혐오 없음)
 - 모델 예측 gender 라벨: 1 (혐오 있음)

[문장] 남자들은 이런애 왜좋아하지?
 - 실제 gender 라벨: 0 (혐오 없음)
 - 모델 예측 gender 라벨: 1 (혐오 있음)

[문장] "디씨 메갈 일베 루리웹 다 ㅈ같은 애들이지 특히 메갈일베는 쌍두마차고"
 - 실제 gender 라벨: 0 (혐오 없음)
 - 모델 예측 gender 라벨: 1 (혐오 있음)



=> 사람이 보기에는 성별 혐오가 분명해 보이는데, 모델이 놓쳤다(FN).
이런 표현을 더 잘 잡기 위해서는 data를 추가하거나 threshold를 조정해야 한다.

=> 이 문장은 비꼼/풍자라서 애매한데, 모델이 성별 혐오로 봤다(FP).
맥락을 더 잘 이해할 수 있는 모델이 필요하다.

In [ ]:
def slice_recall_for_label(label_idx, label_name):
    y_true = labels[:, label_idx]
    y_pred = preds[:, label_idx]

    mask = (y_true == 1)
    if mask.sum() == 0:
        print(f"[{label_name}] 실제 1인 샘플이 없음")
        return

    y_true_pos = y_true[mask]
    y_pred_pos = y_pred[mask]

    recall_slice = (y_pred_pos == 1).mean()
    print(f"[{label_name}] 실제 1 슬라이스 내 Recall: {recall_slice:.3f} (샘플 {len(y_true_pos)}개)")

# 소수자 관련 라벨만 슬라이스 출력
idx_gender = hate_labels.index("gender")
idx_race = hate_labels.index("race")
idx_religion = hate_labels.index("religion")

slice_recall_for_label(idx_gender, "gender")
slice_recall_for_label(idx_race, "race")
slice_recall_for_label(idx_religion, "religion")


[gender] 실제 1 슬라이스 내 Recall: 0.803 (샘플 659개)
[race] 실제 1 슬라이스 내 Recall: 0.417 (샘플 24개)
[religion] 실제 1 슬라이스 내 Recall: 0.907 (샘플 140개)


# 예측 시연


In [ ]:
import torch

def predict_hate(text):
    model = trainer_v2.model
    model.eval()

    enc = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        max_length=128,
    ).to(model.device)

    with torch.no_grad():
        outputs = model(**enc)
        logits = outputs.logits
        probs = torch.sigmoid(logits).cpu().numpy()[0]

    preds = (probs >= 0.5).astype(int)

    res = []
    for label_name, prob, pred in zip(hate_labels, probs, preds):
        res.append({
            "label": label_name,
            "prob": float(prob),
            "pred": int(pred),
        })
    return res

# examples = [
#     "오늘 날씨가 좋아서 산책하러 나갔다 왔어요.",
#     "와 진짜 오늘 일 개빡셌다, 나 완전 녹초 됐어.",
#     "여자들은 감정적이라서 중요한 일에는 맡기면 안 된다.",
#     "이민자들은 다 쫓아내야 나라가 산다.",
#     "장애인들은 회사에 있어도 도움이 안 된다.",
#     "요즘 젊은 애들은 왜 이렇게 기본 예의가 없는지 모르겠다.",
#     "댓글에 “이민자들은 다 쫓아내야 한다”라는 글이 올라와서 충격 받았다.",
#     "여자들이 회사에서 대우 받기 어려운 건 아직 사회가 덜 변한 탓이다",
#     "장애를 가진 사람들도 충분히 일을 잘할 수 있는데, 편견이 여전한 것 같다",
#     "솔직히 외국인들 많아지면 동네 분위기가 좀 바뀌긴 하는 것 같아.",
#     "저 동네는 외국인들이 많아서 위험하니까 가지 마.",
#     "그 게임 서버 또 터졌어? 진짜 짜증나 죽겠네.",
# ]

example = "씨 진짜 짜증나 죽겠네."
predict_hate(example)


[{'label': 'origin', 'prob': 0.00011446417192928493, 'pred': 0},
 {'label': 'physical', 'prob': 0.00012751795293297619, 'pred': 0},
 {'label': 'politics', 'prob': 4.101784361409955e-05, 'pred': 0},
 {'label': 'profanity', 'prob': 0.023929042741656303, 'pred': 0},
 {'label': 'age', 'prob': 0.00017469169688411057, 'pred': 0},
 {'label': 'gender', 'prob': 0.001427947310730815, 'pred': 0},
 {'label': 'race', 'prob': 3.36200901074335e-05, 'pred': 0},
 {'label': 'religion', 'prob': 6.907807983225212e-05, 'pred': 0}]

# 여러 모델 등록 + 비교 함수

In [25]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

# 1) 멀티라벨 라벨 순서 (이미 있으시면 생략)
hate_labels = [
    "origin",
    "physical",
    "politics",
    "profanity",
    "age",
    "gender",
    "race",
    "religion",
]
NUM_LABELS = len(hate_labels)

# 2) tokenizer는 이미 파인튜닝 때 쓰던 것을 그대로 사용
#    (trainer_v2 학습에 사용한 tokenizer 객체)
# tokenizer = ...  # 이미 위에서 정의되어 있다고 가정

# 3) 튜닝 전 Base 모델은 허브에서 바로 불러오기
from transformers import AutoModelForSequenceClassification

base_model = AutoModelForSequenceClassification.from_pretrained(
    "beomi/KcELECTRA-base-v2022",
    num_labels=NUM_LABELS,
    problem_type="multi_label_classification",
)
base_model.to(device)
base_model.eval()

# 4) 튜닝 후 v2 모델은 '이미 학습된 trainer_v2.model' 그대로 사용
v2_model = trainer_v2.model
v2_model.to(device)
v2_model.eval()


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at beomi/KcELECTRA-base-v2022 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(54343, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): L

# 공통 예측 함수 + 비교 함수

In [26]:
def predict_with_model(text: str, model, tokenizer, threshold: float = 0.5):
    enc = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        max_length=128,
    ).to(device)

    with torch.no_grad():
        outputs = model(**enc)
        logits = outputs.logits
        probs = torch.sigmoid(logits).cpu().numpy()[0]

    preds = (probs >= threshold).astype(int)

    results = []
    for label_name, prob, pred in zip(hate_labels, probs, preds):
        results.append({
            "label": label_name,
            "prob": float(prob),
            "pred": int(pred),
        })
    return results


def compare_base_vs_v2(text: str, threshold: float = 0.5):
    base_res = predict_with_model(text, base_model, tokenizer, threshold)
    v2_res   = predict_with_model(text, v2_model,   tokenizer, threshold)

    print("=" * 100)
    print("[입력 문장]")
    print(text)
    print("-" * 100)
    print(f"{'label':<12} {'base_prob':>10} {'base_pred':>10}   {'v2_prob':>10} {'v2_pred':>8}   변화")
    print("-" * 100)

    for b, v in zip(base_res, v2_res):
        label = b["label"]
        bp, bpred = b["prob"], b["pred"]
        vp, vpred = v["prob"], v["pred"]
        changed = "★" if bpred != vpred else ""
        print(f"{label:<12} {bp:10.3f} {bpred:10d}   {vp:10.3f} {vpred:8d}   {changed}")

    print(f"\n※ threshold={threshold:.2f} 기준 (prob ≥ threshold → pred=1)")
    print("=" * 100 + "\n")


# 예시로 비교 실행

In [27]:
examples = [
    "오늘 날씨가 좋아서 산책하러 나갔다 왔어요.",
    "와 진짜 오늘 일 개빡셌다, 나 완전 녹초 됐어.",
    "여자들은 감정적이라서 중요한 일에는 맡기면 안 된다.",
    "이민자들은 다 쫓아내야 나라가 산다.",
    "댓글에 “이민자들은 다 쫓아내야 한다”라는 글이 올라와서 충격 받았다.",
    "나는 이민자들은 다 쫓아내야 나라가 산다는 말에 동의하지 않는다.",
    "남자는 하늘, 여자는 땅.",
    "나는 그가,“남자는 하늘, 여자는 땅“ 이라고 해서 놀랐다.",
    "댓글에 남자는 하늘, 여자는 땅이라는 글이 올이 올라와서 충격 받았다.",
    "그가 말한 성별차이에 대한 발언은 모두를 놀라게했다.",
    "여자는 감성적 성향이 남자보다 강한 편이라는 보고가 있다.",
    "여자는 감성적 성향이 남자보다 강한 편이다.",
]

for text in examples:
    compare_base_vs_v2(text, threshold=0.5)

[입력 문장]
오늘 날씨가 좋아서 산책하러 나갔다 왔어요.
----------------------------------------------------------------------------------------------------
label         base_prob  base_pred      v2_prob  v2_pred   변화
----------------------------------------------------------------------------------------------------
origin            0.505          1        0.000        0   ★
physical          0.509          1        0.000        0   ★
politics          0.479          0        0.000        0   
profanity         0.492          0        0.000        0   
age               0.450          0        0.000        0   
gender            0.525          1        0.000        0   ★
race              0.513          1        0.000        0   ★
religion          0.495          0        0.000        0   

※ threshold=0.50 기준 (prob ≥ threshold → pred=1)

[입력 문장]
와 진짜 오늘 일 개빡셌다, 나 완전 녹초 됐어.
----------------------------------------------------------------------------------------------------
label         base_prob  base_pr

In [ ]:
# Colab 기준: /content 에 폴더가 있다고 가정
!ls

sample_data


In [ ]:
# v2 폴더를 zip으로 압축
!zip -r kmhas_multilabel_full_v2_lr3e-5_ep3_bs32.zip kmhas_multilabel_full_v2_lr3e-5_ep3_bs32

# v3 폴더를 zip으로 압축 (원하시면)
#!zip -r kmhas_multilabel_full_v3_lr2e-5_ep4_bs32.zip kmhas_multilabel_full_v3_lr2e-5_ep4_bs32
